In [7]:
import zipfile
import pandas as pd
import os

# Caminho da pasta que deseja listar os arquivos
caminho_pasta = "Arquivos Diários"

# Listar todos os arquivos na pasta
arquivos = os.listdir(caminho_pasta)

# Mostrar os arquivos
for arquivo in arquivos:
    print(arquivo)


    # Caminho do arquivo zip
    zip_file_path = 'Arquivos Diários/{}'.format(arquivo)

    # Abrir e extrair o conteúdo do arquivo zip
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extrair todos os arquivos
        zip_ref.extractall('arquivos_diarios_extraidos')

        # Ler o arquivo CSV extraído
        # csv_file_path = 'arquivos_diarios_extraidos/{}.csv'.format(arquivo)
        # df = pd.read_csv(csv_file_path)

# Exibir as primeiras linhas do DataFrame
# df.head()

Quanti_20240615.zip
Quanti_20240606.zip
Quanti_20240614.zip
Quanti_20240626.zip
Quanti_20240624.zip
Quanti_20240627.zip
Quanti_20240611.zip
Quanti_20240619.zip
Quanti_20240609.zip
Quanti_20240605.zip
Quanti_20240623.zip
Quanti_20240607.zip
Quanti_20240620.zip
Quanti_20240630.zip
Quanti_20240617.zip
Quanti_20240603.zip
Quanti_20240602.zip
Quanti_20240621.zip
Quanti_20240629.zip
Quanti_20240625.zip
Quanti_20240601.zip
Quanti_20240612.zip
Quanti_20240618.zip
Quanti_20240616.zip
Quanti_20240610.zip
Quanti_20240604.zip
Quanti_20240613.zip
Quanti_20240628.zip
Quanti_20240622.zip
Quanti_20240608.zip


In [9]:
zip_file_path = 'Lote Inicial/{}'.format("LoteInicial.zip")

# Abrir e extrair o conteúdo do arquivo zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extrair todos os arquivos
    zip_ref.extractall('Lote Inicial')

In [141]:
import json  
import os   

def verifica_dados(elem_atual, json_antigo, id):
    # A função verifica_dados compara os dados atuais com os dados antigos.
    elem_antigo = json_antigo[id]  # Obtém o elemento antigo correspondente ao ID

    # Itera sobre cada chave e valor do elemento antigo
    for chave, valor in elem_antigo.items():
        # Compara o valor atual com o valor antigo (convertendo ambos para string)
        if str(elem_atual[chave]) != str(valor):
            # Extrai ISIN e data do ID (que é composto por ISIN e data)
            isin = id.split('_')[0]
            data = id.split('_')[1]
            # Cria um dicionário com os dados a serem registrados
            dados = {
                'ISIN': [isin],
                'campo_alterado': [chave],
                'valor_antigo': [valor],
                'valor_novo': [elem_atual[chave]],
                'data': [data],
            }
            # Verifica se o arquivo CSV já existe
            if os.path.exists('results/serie_valores_substitutos.csv'):
                # Se existir, anexa os novos dados ao arquivo CSV
                pd.DataFrame(dados).to_csv('results/serie_valores_substitutos.csv', mode='a', header=False, index=False)
            else:
                # Se não existir, cria um novo arquivo CSV
                pd.DataFrame(dados).to_csv('results/serie_valores_substitutos.csv', index=False)
            # Atualiza o valor antigo no dicionário com o novo valor
            elem_antigo[chave] = valor
    
    json_antigo[id] = elem_antigo  # Atualiza o dicionário antigo com os valores modificados

    # Salva os dados atualizados em um arquivo JSON
    with open('dados/{}.json'.format(id), 'w') as arquivo_json:
        json.dump(df0, arquivo_json, indent=4, ensure_ascii=False)  # Escreve o dicionário atualizado no arquivo JSON

# Dicionário pode ser consideravelmente grande, no caso de executar com isso em grandes dados
# Fazer um sistema de guardar em arquivos .json, onde é possível verificar as entradas um a um.
for elem in range(len(arquivos)):
    # Lê os arquivos CSV diários extraídos e armazena em um DataFrame
    df1 = pd.read_csv('arquivos_diarios_extraidos/{}'.format(arquivos[elem]))
    # Cria uma nova coluna 'ID' que combina 'ISIN' e 'AuM Fund Date'
    df1['ID'] = df1['ISIN'] + '_' + df1['AuM Fund Date']
    # Remove linhas onde 'ISIN' ou 'ID' são nulos
    df1 = df1.dropna(subset=['ISIN', 'ID'])
    # Remove duplicatas na coluna 'ID', mantendo a última ocorrência
    df1.drop_duplicates('ID', keep='last')  # perde info porém é para dados entre dias

    # Itera sobre cada linha do DataFrame df1
    for _, elem in df1.iterrows():
        df0 = {}  # Inicializa um dicionário vazio para armazenar os dados antigos
        arquivos_dados = os.listdir('dados')  # Lista todos os arquivos no diretório 'dados'

        # Verifica se um arquivo JSON correspondente ao ID já existe
        if str(elem['ID'] + '.json') in arquivos_dados:
            # Se existir, carrega os dados antigos do arquivo JSON
            with open('dados/' + elem['ID'] + '.json', 'r') as arquivo_json:
                df0 = json.load(arquivo_json)
            # Chama a função para verificar os dados
            verifica_dados(elem, df0, elem['ID'])
        else:
            # Se não existir, cria uma nova entrada no dicionário df0
            df0[elem['ID']] = {
                'FE Bid': elem['FE Bid'],
                'AuM Fund': elem['AuM Fund'],
                'AuM Share Class': elem['AuM Share Class'],
                'Bid NAV': elem['Bid NAV'],
                'Ask NAV': elem['Ask NAV'],
                'Valuation NAV': elem['Valuation NAV'],
                'NoS Share Class': elem['NoS Share Class'],
                'NoS Fund': elem['NoS Fund'],
                'FE Offer': elem['FE Offer'],
                'FE NAV': elem['FE NAV'],
                'FE AUM Fund': elem['FE AUM Fund'],
                'FE Generic NAV': elem['FE Generic NAV'],
                'Transaction NAV': elem['Transaction NAV'],
            }
            # Cria um novo arquivo JSON para os dados se não existir
            with open('dados/{}.json'.format(elem['ID']), 'w') as arquivo_json:
                json.dump(df0, arquivo_json, indent=4, ensure_ascii=False)  # Escreve o dicionário no arquivo JSON


KeyboardInterrupt: 